In [1]:
# import packages
import json
import xarray as xr
import dask
import numpy as np
import importlib

import modules
import setup_domain_func

import os

In [ ]:
import logging

In [ ]:
importlib.reload(setup_domain_func)
importlib.reload(modules)

In [2]:
# Get some ressourcers
client, cluster = modules.getCluster('fat', 1, 25)
    
# Do the memory magic...
client.amm.start() 
    
# Write some info about the cluster
print(client.scheduler_info)

<bound method Client.scheduler_info of <Client: 'tcp://172.27.80.110:40000' processes=0 threads=0, memory=0 B>>


In [7]:
client

<Client: 'tcp://172.27.80.110:40000' processes=25 threads=75, memory=745.00 GiB>

In [ ]:
# Read the domain configuration from the respective JSON
with open('conf/domain_config.json', 'r') as j:
    domain_config = json.loads(j.read())

# Read the global configuration from the respective JSON --> Add this as further input parameter
with open('conf/global_config.json', 'r') as j:
    global_config = json.loads(j.read())

# Read the variable configuration from the respective JSON
with open('conf/variable_config.json', 'r') as j:
    variable_config = json.loads(j.read())

In [ ]:
domain_config = domain_config['germany']

In [ ]:
# Setup some logging
logging.basicConfig(filename=f"logs/{domain_config['prefix']}_setup_domain.log", encoding='utf-8', level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

In [ ]:
variable_config = { key:value for key,value in variable_config.items() if key in domain_config['variables']}

In [ ]:
dir_dict = setup_domain_func.set_and_make_dirs(domain_config)

In [ ]:
grd_fle = setup_domain_func.create_grd_file(domain_config, dir_dict)

In [ ]:
syr_calib = domain_config["syr_calib"]
eyr_calib = domain_config["eyr_calib"]

In [ ]:
for month in [7, 10, 11, 12]:
    
    results = []
    
    month_str = str(month).zfill(2)

    for year in range(1988, eyr_calib + 1):
        
        results.append(setup_domain_func.prepare_forecast_dask(domain_config, variable_config, dir_dict, year, month_str))
    
    try:
        dask.compute(results)
        logging.info(f"Slicing for month {month_str} for all years from {syr_calib} to {eyr_calib} successful")
    except:
        logging.warning(f"Something went wrong during slicing for {month_str}")
    

In [ ]:
for month in [7, 6, 9]:
    print(month)